<a href="https://colab.research.google.com/github/pepperamy/others/blob/main/mergesummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from datetime import datetime

In [ ]:
#data from the "historical data"
df = pd.read_csv('ins_2021-03-06-20-44-07-EST-Historical-Report-SP-500-2020-01-01--2021-03-01.csv')

In [ ]:
#df['ins_id'] = df['URL'].apply(lambda x: [i for i in x.split('/') if i != ''][-1])

In [ ]:
df['User Name'].isnull().sum()

0

In [ ]:
df.head()

,Account,User Name,Followers at Posting,Created,Type,Likes,Comments,Views,URL,Link,Photo,Title,Description,Image Text,Sponsor Id,Sponsor Name,Total Interactions,Overperforming Score (weighted — Likes 1x Comments 1x )
0,HCA Healthcare,hcahealthcare,8921.0,2021-02-28 11:38:56 EST,Photo,72,1,0,https://www.instagram.com/p/CL18eAlBgsB/,https://www.instagram.com/p/CL18eAlBgsB/,https://scontent-lax3-1.cdninstagram.com/v/t51...,NaN,Throughout #BlackHistoryMonth we’ve been celeb...,FAIRVII HOS FAIRVIEW PA HOSPITA IRVIEW HO PAR ...,NaN,NaN,73,-1.63
1,Illumina Inc.,illuminainc,5680.0,2021-02-27 17:00:12 EST,Photo,67,0,0,https://www.instagram.com/p/CLz8b8nniw_/,https://www.instagram.com/p/CLz8b8nniw_/,https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,Whole genome sequencing is changing the lives ...,"Working from home, doing remote learning, havi...",NaN,NaN,67,-2.16
2,Humana,humana,10012.0,2021-02-27 10:41:07 EST,Album,219,3,0,https://www.instagram.com/p/CLzRDn-lsen/,https://www.instagram.com/p/CLzRDn-lsen/,https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,"Caregiving may be loving and rewarding work, b...",NaN,NaN,NaN,222,1.55
3,"J.B. Hunt Transport, Inc.",jbhunttransport,1966.0,2021-02-26 17:15:07 EST,Photo,113,5,0,https://www.instagram.com/p/CLxZWNhlifm/,https://www.instagram.com/p/CLxZWNhlifm/,https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,Our parking lot looked a bit different today! ...,NaN,NaN,NaN,118,2.81
4,Gap Inc.,gapinc,28387.0,2021-02-26 15:36:58 EST,Photo,428,11,0,https://www.instagram.com/p/CLxOHlwH-V8/,https://www.instagram.com/p/CLxOHlwH-V8/,https://scontent-sea1-1.cdninstagram.com/v/t51...,NaN,We’re answering the call from the White House ...,We're answering the call from the White House ...,NaN,NaN,439,1.96


In [ ]:
from dateutil import parser

In [ ]:
# change the date type
df.Created = df.Created.apply(lambda x: parser.parse(x))

C:\Users\KK\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1218: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
C:\Users\KK\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1218: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [ ]:
result_1 = df.groupby(['User Name'])['User Name'].count()

In [ ]:
result_1 = result_1.to_frame()

In [ ]:
result_1 = result_1.rename(columns = {'User Name':'count'})

In [ ]:
result_1.head()

,count
User Name,
3m,13
abbottglobal,24
abbvie,4
accenture,1
adobe,2


In [ ]:
result_2 = df.groupby(['User Name'])['Total Interactions','Comments','Views','Likes'].sum() # the column name is different from facebook data and the ins data
                                                 #need to change the column name

C:\Users\KK\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [ ]:
result_2.head()

,Comments,Views,Likes
User Name,,,
3m,3190,75396,30667
abbottglobal,2789,122160,47773
abbvie,60,2122,2520
accenture,18,0,2475
adobe,163,0,5760


In [ ]:
result = result_1.merge(result_2, right_index =True, left_index = True)

In [ ]:
result = result.reset_index()

In [ ]:
result.head()

,User Name,count,Comments,Views,Likes
0,3m,13,3190,75396,30667
1,abbottglobal,24,2789,122160,47773
2,abbvie,4,60,2122,2520
3,accenture,1,18,0,2475
4,adobe,2,163,0,5760


In [ ]:
#get the start date and end date of each account
def get_date(x):
    global df
    temp = df[df['User Name'] == x]
    temp_sort = temp.sort_values(by = ['Created'])
    sd = temp_sort.Created.iloc[0]
    ed = temp_sort.Created.iloc[-1]
    return sd, ed

In [ ]:
result['start_date'] = result['User Name'].apply(lambda x:get_date(x)[0])

In [ ]:
result['end_date'] = result['User Name'].apply(lambda x:get_date(x)[1])

In [ ]:
result.head()

,User Name,count,Comments,Views,Likes,start_date,end_date
0,3m,13,3190,75396,30667,2020-03-20 15:04:00,2020-11-09 13:14:30
1,abbottglobal,24,2789,122160,47773,2020-03-18 20:08:04,2021-02-02 11:07:28
2,abbvie,4,60,2122,2520,2020-03-26 08:30:08,2020-12-21 08:00:10
3,accenture,1,18,0,2475,2020-05-06 10:28:49,2020-05-06 10:28:49
4,adobe,2,163,0,5760,2020-04-15 12:39:38,2020-07-09 15:13:20


In [ ]:
#data from the leaderboad
leaderboard = pd.read_csv('ins_2021-03-06-20-45-23-EST-instagram_leaderboard_S&P-500_2020-01-01_00-00-00_ET_2021-02-28_23-59-59_ET.csv')

In [ ]:
leaderboard.shape

(422, 32)

In [ ]:
leaderboard.head()

,Account,Codename,URL,Total Interactions,Likes,Comments,Views,All Interaction Rate,Albums,Photos,...,Album Comments,Photo Comments,Video Comments,IGTV Comments,Album Video Views,Video Video Views,IGTV Video Views,Followers,Follower Growth,Follower Growth %
0,Disney,disney,https://www.instagram.com/disney/,"205,387,804","204,377,116","1,010,688","234,110,605",0.69%,1.05%,1.02%,...,"121,075","537,751","242,080","109,782","9,673,381","128,031,228","96,405,996","29,132,077","6,757,337",30.2%
1,apple,apple,https://www.instagram.com/apple/,"47,927,910","47,620,270","307,640","44,416,024",1.19%,1.19%,1.38%,...,"158,821","52,404","27,392","69,023","14,107,341","15,510,082","14,798,601","25,278,639","4,121,248",19.48%
2,Starbucks Coffee ☕,starbucks,https://www.instagram.com/starbucks/,"26,980,816","26,614,593","366,223","17,294,301",0.62%,0.63%,0.67%,...,"34,923","283,426","47,874",0,"641,127","16,653,174",0,"18,072,087","-397,128",-2.15%
3,Monster Energy,monsterenergy,https://www.instagram.com/monsterenergy/,"23,947,962","23,840,694","107,268","111,976,146",0.43%,0.58%,0.8%,...,"18,631","28,924","54,477","5,236","11,907,998","92,295,543","7,772,605","7,389,883","1,060,279",16.75%
4,Nike,nike,https://www.instagram.com/nike/,"21,527,364","21,285,473","241,891","74,921,239",0.51%,0.33%,0.68%,...,"43,713","66,535","121,733","9,910",0,"66,130,830","8,790,409","136,112,965","38,318,623",39.18%


In [ ]:
leaderboard_1 = leaderboard.loc[:,['Codename','URL','Total Posts','Followers','Total Interactions','Likes','Comments','Views']]

In [ ]:
#leaderboard_1['id'] = leaderboard_1.URL.apply(lambda x: x.split('/')[-1] if (type(x) == str) else x)

In [ ]:
#leaderboard_1[leaderboard_1['id'].isnull()]

In [ ]:
#convert the type of id (float)
#result['Facebook Id'] = result['Facebook Id'].apply(lambda x: str(x))

In [ ]:
result_merge =pd.merge(result, leaderboard_1,left_on = ['User Name'], right_on = ['Codename'],how = 'right' )

In [ ]:
result_merge

,User Name,count,Comments_x,Views_x,Likes_x,start_date,end_date,Codename,URL,Total Posts,Followers,Total Interactions,Likes_y,Comments_y,Views_y
0,NaN,NaN,NaN,NaN,NaN,NaT,NaT,disney,https://www.instagram.com/disney/,"1,158","29,132,077","205,387,804","204,377,116","1,010,688","234,110,605"
1,apple,2.0,4258.0,0.0,1010454.0,2020-07-26 11:53:58,2020-11-02 11:29:26,apple,https://www.instagram.com/apple/,174,"25,278,639","47,927,910","47,620,270","307,640","44,416,024"
2,starbucks,2.0,611.0,0.0,116897.0,2020-11-24 11:00:09,2020-12-15 11:00:50,starbucks,https://www.instagram.com/starbucks/,239,"18,072,087","26,980,816","26,614,593","366,223","17,294,301"
3,monsterenergy,2.0,191.0,297751.0,38919.0,2020-04-24 13:57:39,2020-04-25 12:00:28,monsterenergy,https://www.instagram.com/monsterenergy/,820,"7,389,883","23,947,962","23,840,694","107,268","111,976,146"
4,NaN,NaN,NaN,NaN,NaN,NaT,NaT,nike,https://www.instagram.com/nike/,36,"136,112,965","21,527,364","21,285,473","241,891","74,921,239"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,NaN,NaN,NaN,NaN,NaN,NaT,NaT,vfcorp,https://www.instagram.com/vfcorp/,No posts in this timeframe,0,0,0,0,0
418,NaN,NaN,NaN,NaN,NaN,NaT,NaT,ross.stores,https://www.instagram.com/ross.stores/,No posts in this timeframe,0,0,0,0,0
419,NaN,NaN,NaN,NaN,NaN,NaT,NaT,realtyincome,https://www.instagram.com/realtyincome/,No posts in this timeframe,0,0,0,0,0
420,NaN,NaN,NaN,NaN,NaN,NaT,NaT,moodyscorporation,https://www.instagram.com/moodyscorporation/,No posts in this timeframe,0,0,0,0,0


In [ ]:
result_merge.to_excel('ins_summary_posts_2020-01-01--2021-02-28.xlsx',index = False)